<a href="https://colab.research.google.com/github/LxYuan0420/eat_tensorflow2_in_30_days/blob/master/notebooks/4_3_Rules_of_Using_the_AutoGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

There are three ways of constructing graph: static, dynamic and Autograph.

TensorFlow 2.X uses dynamic graph and Autograph.

Dynamic graph is easier for debugging with higher encoding efficiency, but with lower efficiency in execution.

Static graph has high efficiency in execution, but more difficult for debugging.

Autograph mechanism transforms dynamic graph into static graph, making allowance for both executing and encoding efficiencies.

There are certain rules for the code that is able to converted by Autograph, or it could result in failure or unexpected results.

We are going to introduce the coding rules of Autograph and its mechanism of converting into static graph, together with introduction about how to construct Autograph using tf.Module.

This section introduce the coding rules of using Autograph. We will introduce the mechanisms of Autograph in next section and explain the logic behind the rules there.

**1. Summarization of the Coding Rules of Autograph**

- We should use the TensorFlow-defined functions to be decorated by @tf.function as much as possible, instead of those Python functions. For instance, tf.print should be used instead of print; tf.range should be used instead of range; tf.constant(True) should be used instead of True.
- Avoid defining tf.Variable inside the decorator @tf.function.
- Functions that are decorated by @tf.function cannot modify the struct data types variables outside the function such as Python list, dictionary, etc.

**2. Explanations to the Autograph Coding Rules**

**2.1 We should use the TensorFlow-defined functions to be decorated by @tf.function as much as possible, instead of those Python functions.**



In [2]:
import numpy as np
import tensorflow as tf

In [3]:
@tf.function
def np_random():
    a = np.random.randn(3,3)
    tf.print(a)

@tf.function
def tf_random():
    a = tf.random.normal((3,3))
    tf.print(a)

In [4]:
# same results after each execution of np_random
np_random()
np_random()

array([[ 1.04636729,  0.7851914 ,  0.47463141],
       [-0.28886157,  0.0997656 ,  1.02688895],
       [ 1.11276276, -1.15013536, -0.86116777]])
array([[ 1.04636729,  0.7851914 ,  0.47463141],
       [-0.28886157,  0.0997656 ,  1.02688895],
       [ 1.11276276, -1.15013536, -0.86116777]])


In [5]:
# new random numbers are generated after each eecution of tf_random
tf_random()
tf_random()

[[0.350216538 0.181701332 1.00079453]
 [0.450795293 0.588276207 0.968606174]
 [-1.8077414 1.42580962 -0.598237872]]
[[-0.5538432 -1.33888221 -0.0170988608]
 [0.240584984 -0.211996689 0.375466734]
 [-0.689313412 1.06698382 1.36646891]]


**2.2 Avoid defining tf.Variable inside the decorator @tf.function.**

In [6]:
x = tf.Variable(1.0, dtype=tf.float32)

@tf.function
def outer_var():
    x.assign_add(1.0)
    tf.print(x)
    return (x)

outer_var()
outer_var()

2
3


<tf.Tensor: shape=(), dtype=float32, numpy=3.0>

In [7]:
@tf.function
def inner_var():
    x = tf.Variable(1.0, dtype=tf.float32)
    x.assign_add(1.0)
    tf.print(x)
    return (x)

inner_var()
inner_var()

ValueError: ignored

**2.3 Functions that are decorated by @tf.function cannot modify the struct data types variables outside the function such as Python list, dictionary, etc.**

In [8]:
tensor_list = []

#@tf.function #autograph will result in something unexpected if executing this line
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list

append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))

print(tensor_list)

[<tf.Tensor: shape=(), dtype=float32, numpy=5.0>, <tf.Tensor: shape=(), dtype=float32, numpy=6.0>]


In [10]:
tensor_list = []

@tf.function 
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list

append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))

print(tensor_list)

[<tf.Tensor 'x:0' shape=() dtype=float32>]


Summary: 

Functions decorated as `tf.function` can only update tf.Variables that are defined outside of the function. 

`tf.function` decorated function cant modify python data struct (doesn' matter if it is defined inside or outside) 